In [1]:
"""
Notebook for streaming data from a microphone in realtime

audio is captured using pyaudio
then converted from binary data to ints using struct
then displayed using matplotlib

scipy.fftpack computes the FFT

if you don't have pyaudio, then run

>>> pip install pyaudio

note: with 2048 samples per chunk, I'm getting 20FPS
when also running the spectrum, its about 15FPS
"""

import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024*2              # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

In [2]:
# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# format spectrum axes
ax2.set_xlim(20, RATE / 2)

print('stream started')
chosen_frequencies = [100, 1000, 5000]
# Indices corresponding to chosen frequencies
chosen_indices = [np.abs(xf - freq).argmin() for freq in chosen_frequencies]
# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    line.set_ydata(data_np)
    
    # compute FFT and update line
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (128 * CHUNK))
    # x=(np.abs(yf[0:CHUNK])  / (128 * CHUNK))
    # print(x.shape)
    # update figure canvas
    # for freq, value in zip(xf, np.abs(yf[0:CHUNK]) / (128 * CHUNK)):
    #     print(f"Frequency: {freq:.2f} Hz, Value: {value:.2f}")
    for freq, index in zip(chosen_frequencies, chosen_indices):
        value = np.abs(yf[index]) / (128 * CHUNK)
        print(f"Frequency: {freq} Hz, Value: {value}")
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

stream started
Frequency: 100 Hz, Value: 0.004899254298260441
Frequency: 1000 Hz, Value: 0.04182645529457244
Frequency: 5000 Hz, Value: 0.015677268594166432


C:\Users\rajka\AppData\Local\Temp\ipykernel_16084\1937994783.py:55: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 255 to int8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  data_np = np.array(data_int, dtype='b')[::2] + 128


Frequency: 100 Hz, Value: 0.004025187424473362
Frequency: 1000 Hz, Value: 0.04169896252909959
Frequency: 5000 Hz, Value: 0.014871631181297265
Frequency: 100 Hz, Value: 0.48136016869687553
Frequency: 1000 Hz, Value: 0.026970191427315562
Frequency: 5000 Hz, Value: 0.013052617287753572
Frequency: 100 Hz, Value: 0.4119957644542971
Frequency: 1000 Hz, Value: 0.04695871113339655
Frequency: 5000 Hz, Value: 0.027555342826200528
Frequency: 100 Hz, Value: 0.7977706725802635
Frequency: 1000 Hz, Value: 0.036829569820919776
Frequency: 5000 Hz, Value: 0.01365456634011832


C:\Users\rajka\AppData\Local\Temp\ipykernel_16084\1937994783.py:55: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 254 to int8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  data_np = np.array(data_int, dtype='b')[::2] + 128
C:\Users\rajka\AppData\Local\Temp\ipykernel_16084\1937994783.py:55: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 250 to int8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  data_np = np.array(data_int, dtype='b')[::2] + 128
C:\Users\rajka\AppData\Local\Temp\ipykernel_16084\1937994783.py:55: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 246 

Frequency: 100 Hz, Value: 0.13518712161300853
Frequency: 1000 Hz, Value: 0.019019755243018556
Frequency: 5000 Hz, Value: 0.036548291154670016
Frequency: 100 Hz, Value: 0.40150539852274125
Frequency: 1000 Hz, Value: 0.10288140554623044
Frequency: 5000 Hz, Value: 0.024232643984945162
Frequency: 100 Hz, Value: 0.43093650619779955
Frequency: 1000 Hz, Value: 0.04445490219524632
Frequency: 5000 Hz, Value: 0.03110065852644318


...

Frequency: 100 Hz, Value: 0.5342187225215798
Frequency: 1000 Hz, Value: 0.03393660470299343
Frequency: 5000 Hz, Value: 0.01424512364115008
Frequency: 100 Hz, Value: 0.27611227865348786
Frequency: 1000 Hz, Value: 0.050287398341294275
Frequency: 5000 Hz, Value: 0.021139011979153827
Frequency: 100 Hz, Value: 0.5759212032083249
Frequency: 1000 Hz, Value: 0.03410762621837604
Frequency: 5000 Hz, Value: 0.0095029750558973
Frequency: 100 Hz, Value: 0.22870477639837153
Frequency: 1000 Hz, Value: 0.044818800719655244
Frequency: 5000 Hz, Value: 0.012612618554055715
Frequency: 100 Hz, Value: 0.269582307505537
Frequency: 1000 Hz, Value: 0.07562561999973844
Frequency: 5000 Hz, Value: 0.02112223436233375
Frequency: 100 Hz, Value: 0.6013054391405259
Frequency: 1000 Hz, Value: 0.10087838889288071
Frequency: 5000 Hz, Value: 0.020815157072518877
Frequency: 100 Hz, Value: 0.4438179894684883
Frequency: 1000 Hz, Value: 0.12311613116601912
Frequency: 5000 Hz, Value: 0.015216381426261751
Frequency: 100 Hz, Va

C:\Users\rajka\AppData\Local\Temp\ipykernel_16084\1937994783.py:55: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 198 to int8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  data_np = np.array(data_int, dtype='b')[::2] + 128
C:\Users\rajka\AppData\Local\Temp\ipykernel_16084\1937994783.py:55: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 188 to int8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  data_np = np.array(data_int, dtype='b')[::2] + 128
C:\Users\rajka\AppData\Local\Temp\ipykernel_16084\1937994783.py:55: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 177 

Frequency: 100 Hz, Value: 0.16423326967944016
Frequency: 1000 Hz, Value: 0.009098785687370737
Frequency: 5000 Hz, Value: 0.03333473873851256
Frequency: 100 Hz, Value: 0.12861417363168287
Frequency: 1000 Hz, Value: 0.023212332979738445
Frequency: 5000 Hz, Value: 0.0517519282633845
Frequency: 100 Hz, Value: 0.31949808242298866
Frequency: 1000 Hz, Value: 0.145577003078684
Frequency: 5000 Hz, Value: 0.04864530778779087
Frequency: 100 Hz, Value: 0.16701252001340824
Frequency: 1000 Hz, Value: 0.1036269358528543
Frequency: 5000 Hz, Value: 0.011888815471386658


C:\Users\rajka\AppData\Local\Temp\ipykernel_16084\1937994783.py:55: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 175 to int8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  data_np = np.array(data_int, dtype='b')[::2] + 128
C:\Users\rajka\AppData\Local\Temp\ipykernel_16084\1937994783.py:55: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 162 to int8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  data_np = np.array(data_int, dtype='b')[::2] + 128
C:\Users\rajka\AppData\Local\Temp\ipykernel_16084\1937994783.py:55: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 160 

Frequency: 100 Hz, Value: 0.3120489543761627
Frequency: 1000 Hz, Value: 0.029184034735076688
Frequency: 5000 Hz, Value: 0.013777293626301281
Frequency: 100 Hz, Value: 0.1996329146420177
Frequency: 1000 Hz, Value: 0.025883352921560833
Frequency: 5000 Hz, Value: 0.0205015120361757
Frequency: 100 Hz, Value: 0.18616398929311784
Frequency: 1000 Hz, Value: 0.07271970574754634
Frequency: 5000 Hz, Value: 0.009933302005359265
Frequency: 100 Hz, Value: 0.40684037209319474
Frequency: 1000 Hz, Value: 0.023678871736050856
Frequency: 5000 Hz, Value: 0.016969087867545973
Frequency: 100 Hz, Value: 0.1887435005179638
Frequency: 1000 Hz, Value: 0.051251425213146226
Frequency: 5000 Hz, Value: 0.0019068920644078862
Frequency: 100 Hz, Value: 0.4978569249380963
Frequency: 1000 Hz, Value: 0.019143522564417887
Frequency: 5000 Hz, Value: 0.01563137059323045
Frequency: 100 Hz, Value: 0.31611756214721237
Frequency: 1000 Hz, Value: 0.11110223922155293
Frequency: 5000 Hz, Value: 0.014503570879094124
Frequency: 100